In [1]:
from gurobipy import *
import numpy as np
import string
import random

## Precision LP

In [12]:
S = [0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]

dic = list(string.ascii_lowercase)
E2words = {}

for i in range(10):
    E2words[i] = set(random.sample(dic, 5))

            
Email, S = multidict({i: [S[i]] for i in range(10)})

print("=====================")
print(E2words)
print(dic)
print(Email)

{0: set(['e', 'z', 'm', 'd', 'w']), 1: set(['p', 'e', 'l', 'g', 'v']), 2: set(['q', 'r', 'u', 't', 'f']), 3: set(['i', 'y', 'z', 'e', 'o']), 4: set(['p', 'r', 'j', 'l', 'o']), 5: set(['i', 'h', 'k', 'r', 'l']), 6: set(['r', 'j', 'f', 'g', 'n']), 7: set(['q', 'x', 'c', 'r', 'f']), 8: set(['q', 'r', 'b', 'k', 'g']), 9: set(['y', 'h', 'b', 'q', 'g'])}
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [15]:
model = Model('precision')

# Email variable
w = model.addVars(Email, lb=0.0, ub =1.0, vtype='C', name = 'Weight')
I = model.addVars(Email, lb=0.0, ub =1.0, vtype='B', name = 'Indication')
u = model.addVar(lb=0.0, ub =1.0, vtype='C', name = 'Sum_Indication')

# keyword field
Iw = model.addVars(dic, vtype='B', name = 'word_Indication')

In [16]:
# Weight-Sum constraints
model.addConstr(quicksum(w[i] for i in range(10)) == 1.0, 'Sum(w[i]) == 1.0')

for i in Email:
    model.addConstr(w[i] <= u, 'w[i] <= u')
    model.addConstr(w[i] <= 1e4*I[i], 'w[i] <= M*I(i)')
    model.addConstr(w[i] >= u-1e4*(1-I[i]),'w[i] >= u - M*[1-I(i)]')
    model.addConstr(w[i] >= 0.0, 'Weight-positive')
    
    model.addGenConstrOr(I[i], [Iw[word] for word in E2words[i]], "keyword_Email_Con")
        
model.setObjective(w.prod(S), GRB.MAXIMIZE)
    
model.optimize()

#if model.Status == GRB.OPTIMAL:
#    print(z.X)
print('===================')
for v in model.getVars():
    print(v.varName, v.x)
print('===================')
print('Obj:', model.objVal)
model.write("precision.lp")


Optimize a model with 41 rows, 47 columns and 90 nonzeros
Model has 10 general constraints
Variable types: 11 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve added 33 rows and 0 columns
Presolve removed 0 rows and 12 columns
Presolve time: 0.00s
Presolved: 74 rows, 35 columns, 186 nonzeros
Variable types: 11 continuous, 24 integer (24 binary)
Found heuristic solution: objective -0.0000000

Root relaxation: objective 1.000000e+00, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       1.0000000    1.00000  0.00%     -    0s

Explored 0 nodes (12 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 1 -0 

Optimal solution found (

## Recall LP

In [6]:
S = [0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
u = 1/sum(S)
#print(u)

dic = list(string.ascii_lowercase)
E2words = {}
#word2E = {}

for i in range(10):
    E2words[i] = set(random.sample(dic, 5))


#for kw in dic:
#    word2E[kw] = []
#    for i in range(10):
#        if kw in E2words[i]: 
#            word2E[kw].append(i) 
            
Email, S, words = multidict({i: [S[i], E2words[i]] for i in range(10)})

print(words)
print("=====================")
print(E2words)

{0: set(['q', 'p', 'z', 'm', 'n']), 1: set(['a', 'h', 'b', 'i', 'g']), 2: set(['r', 'b', 'e', 's', 'v']), 3: set(['i', 'h', 'd', 'g', 'f']), 4: set(['s', 'r', 'k', 'w', 'j']), 5: set(['y', 'x', 'k', 'v', 'n']), 6: set(['x', 'c', 'l', 'w', 'v']), 7: set(['k', 'b', 'r', 's', 'v']), 8: set(['a', 'h', 'k', 'j', 'd']), 9: set(['a', 'e', 'j', 'm', 'v'])}
{0: set(['q', 'p', 'z', 'm', 'n']), 1: set(['a', 'h', 'b', 'i', 'g']), 2: set(['r', 'b', 'e', 's', 'v']), 3: set(['i', 'h', 'd', 'g', 'f']), 4: set(['s', 'r', 'k', 'w', 'j']), 5: set(['y', 'x', 'k', 'v', 'n']), 6: set(['x', 'c', 'l', 'w', 'v']), 7: set(['k', 'b', 'r', 's', 'v']), 8: set(['a', 'h', 'k', 'j', 'd']), 9: set(['a', 'e', 'j', 'm', 'v'])}


In [24]:
model = Model('Recall')

# Email variable
w = model.addVars(Email, lb=0.0, ub =1.0, vtype='C', name = 'Weight')
Ie = model.addVars(Email, lb=0.0, ub =1.0, vtype='B', name = 'email_Indication')

# keyword field
Iw = model.addVars(dic, vtype='B', name = 'word_Indication')

In [25]:
for i in Email:
    model.addConstr(w[i] <= u, 'w[i] <= u')
    model.addConstr(w[i] <= 1e4*Ie[i], 'w[i] <= M*I(i)')
    model.addConstr(w[i] >= u-1e4*(1-Ie[i]),'w[i] >= u - M*[1-I(i)]')
    model.addConstr(w[i] >= 0.0, 'Weight-positive')
    
    model.addGenConstrOr(Ie[i], [Iw[word] for word in E2words[i]], "keyword_Email_Con")
        
model.setObjective(w.prod(S), GRB.MAXIMIZE)

model.optimize()

#if model.Status == GRB.OPTIMAL:
#    print(z.X)
print('===================')
for v in model.getVars():
    print(v.varName, v.x)
print('===================')
print('Obj:', model.objVal)
model.write("recall.lp")


Optimize a model with 40 rows, 46 columns and 60 nonzeros
Model has 10 general constraints
Variable types: 10 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 1e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 40 rows and 46 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 1 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
('Weight[0]', 0.2000000000000008)
('Weight[1]', 0.2000000000000008)
('Weight[2]', 0.2000000000000008)
('Weight[3]', 0.2000000000000008)
('Weight[4]', 0.2000000000000008)
('Weight[5]', 0.2000000000000008)
('Weight[6]', 0.2000000000000008)
('Weight[7]', 0.2000000000000008)
('Weight[8]', 0.20000000

## F1-Score LP

In [39]:
S = [0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]

C = sum(S)
#print(c)

            
Email, S = multidict({i: [S[i]] for i in range(10)})

E2words={0: set(['y', 'c', 't', 'o', 'f']), 1: set(['c', 'k', 'z', 'd', 'g']), 2: set(['i', 'm', 'e', 't', 'v']), 3: set(['h', 'c', 'm', 'w', 'v']), 4: set(['p', 'k', 'l', 'd', 'x']), 5: set(['n', 'c', 'f', 'l', 'v']), 6: set(['b', 'j', 'e', 'g', 'n']), 7: set(['i', 'x', 'u', 'g', 'o']), 8: set(['q', 'b', 'e', 'g', 'w']), 9: set(['y', 'c', 'b', 'o', 'v'])}
word2E={ 'c': [0, 1, 3, 5, 9], 'b': [6, 8, 9], 'e': [2, 6, 8], 'd': [1, 4], 'g': [1, 6, 7, 8], 'f': [0, 5], 'i': [2, 7], 'h': [3], 'k': [1, 4], 'j': [6], 'm': [2, 3], 'l': [4, 5], 'o': [0, 7, 9], 'n': [5, 6], 'q': [8], 'p': [4],  'u': [7], 't': [0, 2], 'w': [3, 8], 'v': [2, 3, 5, 9], 'y': [0, 9], 'x': [4, 7], 'z': [1]}

dic=['b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 't', 'u', 'v', 'w', 'x', 'y', 'z']

print(dic)
print(words)
print("=====================")
print(word2E)

f1_t = (2*C/len(S))/(1+C/len(S))

['b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 't', 'u', 'v', 'w', 'x', 'y', 'z']
{0: set(['q', 'p', 'z', 'm', 'n']), 1: set(['a', 'h', 'b', 'i', 'g']), 2: set(['r', 'b', 'e', 's', 'v']), 3: set(['i', 'h', 'd', 'g', 'f']), 4: set(['s', 'r', 'k', 'w', 'j']), 5: set(['y', 'x', 'k', 'v', 'n']), 6: set(['x', 'c', 'l', 'w', 'v']), 7: set(['k', 'b', 'r', 's', 'v']), 8: set(['a', 'h', 'k', 'j', 'd']), 9: set(['a', 'e', 'j', 'm', 'v'])}
{'c': [0, 1, 3, 5, 9], 'b': [6, 8, 9], 'e': [2, 6, 8], 'd': [1, 4], 'g': [1, 6, 7, 8], 'f': [0, 5], 'i': [2, 7], 'h': [3], 'k': [1, 4], 'j': [6], 'm': [2, 3], 'l': [4, 5], 'o': [0, 7, 9], 'n': [5, 6], 'q': [8], 'p': [4], 'u': [7], 't': [0, 2], 'w': [3, 8], 'v': [2, 3, 5, 9], 'y': [0, 9], 'x': [4, 7], 'z': [1]}


In [40]:
model = Model('F1')

# Email variable
w = model.addVars(Email, lb=0.0, ub =1.0, vtype='C', name = 'Weight')
Ie = model.addVars(Email, lb=0.0, ub =1.0, vtype='B', name = 'Indication')
u = model.addVar(lb=0.0, ub =1.0, vtype='C', name = 'Sum_Indication')

# keyword field
Iw = model.addVars(dic, vtype='B', name = 'word_Indication')

In [41]:
model.addConstr(quicksum(w[i] for i in range(10))+u*C == 1.0, 'sum(w[i])+uC=1.0')

for i in Email:
    model.addConstr(w[i] <= u, "w["+str(i)+"]<=u")
    model.addConstr(w[i] <= 1e4*Ie[i], "w["+str(i)+"]<=M*I("+str(i)+")")
    model.addConstr(w[i] >= u-1e4*(1-Ie[i]),"w["+str(i)+"]>=u-M*[1-I("+str(i)+")]")
    model.addConstr(w[i] >= 0.0, "Weight-positive["+str(i)+"]")
    
    model.addGenConstrOr(Ie[i], [Iw[word] for word in E2words[i]], "keyword_Email_Con["+str(i)+"]")
        
model.setObjective(w.prod(S), GRB.MAXIMIZE)
    
model.optimize()

#if model.Status == GRB.OPTIMAL:
#    print(z.X)
print('===================')
for v in model.getVars():
    print(v.varName, v.x)
print('===================')
print('Obj:', model.objVal*2)

print(f1_t)
model.write("f1-score.lp")


Optimize a model with 41 rows, 44 columns and 91 nonzeros
Model has 10 general constraints
Variable types: 11 continuous, 33 integer (33 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve added 26 rows and 0 columns
Presolve removed 0 rows and 11 columns
Presolve time: 0.00s
Presolved: 67 rows, 33 columns, 169 nonzeros
Variable types: 11 continuous, 22 integer (22 binary)
Found heuristic solution: objective -0.0000000

Root relaxation: objective 5.000000e-01, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.5000000    0.50000  0.00%     -    0s

Explored 0 nodes (13 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 0.5 -0 

Optimal solution found